# LightGBM

In [1]:
import pandas as pd
test_path = "/home/joseph/datasets/earthquake_damage_forecasting/test_data_embeds.csv"
test_df = pd.read_csv(test_path)
test_df.head()

,Unnamed: 0,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16
0,0,300051,3,20,7,6,0,1,0,0,...,0.674943,1.233606,1.074826,0.948696,1.789101,-0.690246,1.215929,0.641817,1.489536,-1.748435
1,1,99355,2,25,13,5,0,1,0,0,...,-0.633836,0.383985,0.636826,0.662928,1.637369,-0.900778,1.354285,0.221195,0.662908,0.386138
2,2,890251,2,5,4,5,0,1,0,0,...,-0.570019,0.412793,1.512381,-0.302350,0.350699,-1.007432,1.000211,0.475425,-0.184000,-0.991295
3,3,745817,1,0,19,3,0,0,0,0,...,-1.826422,2.925866,2.126571,1.424021,2.839799,-1.503001,1.168797,2.822999,0.984964,0.840190
4,4,421793,3,15,8,7,0,1,0,0,...,-0.622701,1.714811,1.518751,0.280875,1.983836,-0.291633,1.130178,0.390867,1.355002,-1.796739


In [2]:
import numpy as np
def preprocess_embeds_df(df: pd.DataFrame) -> np.ndarray:
    """Assumes that df already contains the embeds. Processes the df and
    converts it to a numpy array to be used in self.predict.
    """
    if "building_id" in df.columns:
        df = df.drop(["building_id"], axis=1)

    if "Unnamed: 0" in df.columns:
        df = df.drop(["Unnamed: 0"], axis=1)

    return np.array(df)


In [3]:
test = preprocess_embeds_df(test_df)
test.shape

(86868, 81)

In [4]:
test[0][None].shape

(1, 81)

In [1]:
import lightgbm as lgb
model_path = "/home/joseph/Coding/ml_projects/earthquake_forecasting/models/lgb/model0.txt"
model = lgb.Booster(model_file=model_path)

In [6]:
model.num_feature()

81

In [11]:
model.feature_importance()

array([ 2298,  9425, 10043,  6319,   569,  1237,   438,   356,   634,
         776,  1235,   735,   582,   247,   322,  2078,  1008,   598,
         521,   200,    53,    68,   110,    41,    52,     0,   177,
         825,   307,   876,   141,   375,   964,   671,   536,  1334,
        1110,   535,   967,    78,   679,   697,   107,   693,  1337,
         395,  1068,   577,   150,  1380,  1074,    75,    76,   475,
          44,    59,    10,    54,   252,    49,   228,   254,   120,
         400,   164,  8098,  8318,  8108,  8257,  8212,  8333,  8066,
        7712,  8390,  8016,  7896,  7830,  7416,  7947,  7876,  7839],
      dtype=int32)

In [21]:
import multiprocessing
multiprocessing.cpu_count()

8

In [24]:
y_pred = model.predict(test[0][None], num_threads=1)
y_pred.shape

(1, 3)

In [28]:
from multiprocessing import Pool
import tqdm
pool = Pool(processes=8)
def predict(_id):
    print(f"Predicting for {_id}\n")
    y_pred = model.predict(test[0][None], num_threads=1)
    return _id

# starttime=time.time()
# pool = Pool(2)
# multiple_results = [pool.apply_async(functionToParallize) for i in [2,3]]
# p=[res.get() for res in multiple_results]
# print((time.time()-starttime)/60)

results = [pool.apply_async(predict, args=(x,)) for x in range(8)]
p = [res.get() for res in results]
print(p)
pool.close()
pool.join()

Predicting for 4
Predicting for 2
Predicting for 7
Predicting for 3
Predicting for 0
Predicting for 1

Predicting for 5

Predicting for 6


In [30]:
from multiprocessing import Pool
import tqdm
pool = Pool(processes=8)
def predict(_id):
    print(f"Predicting for {_id}\n")
    y_pred = model.predict(test[0][None], num_threads=1)
    return _id

results = [pool.apply(predict, args=(x,)) for x in range(8)]
print(results)
pool.close()
pool.join()

In [19]:
out

[]

In [ ]:
y_pred = model.predict(test[0][None])
y_pred.shape

In [16]:
# Probabilities for each class for a single sample
y_pred

array([[0.00084939, 0.26037801, 0.7387726 ]])

In [19]:
damage = y_pred.argmax(axis=1)+1
damage

array([3])

2 samples

In [20]:
y_pred = model.predict(test[:2])
y_pred.shape

(2, 3)

In [21]:
y_pred

array([[8.49388578e-04, 2.60378007e-01, 7.38772605e-01],
       [2.98472630e-03, 9.26326325e-01, 7.06889488e-02]])

In [22]:
damage = y_pred.argmax(axis=1)+1
damage

array([3, 2])

# API Tests

In [116]:
ex = {
  "building_id": 802906,
  "geo_level_1_id": 6,
  "geo_level_2_id": 487,
  "geo_level_3_id": 12198,
  "count_floors_pre_eq": 2,
  "age": 30,
  "area_percentage": 6,
  "height_percentage": 5,
  "land_surface_condition": "t",
  "foundation_type": "r",
  "roof_type": "n",
  "ground_floor_type": "f",
  "other_floor_type": "q",
  "position": "t",
  "plan_configuration": "d",
  "has_superstructure_adobe_mud": 1,
  "has_superstructure_mud_mortar_stone": 1,
  "has_superstructure_stone_flag": 0,
  "has_superstructure_cement_mortar_stone": 0,
  "has_superstructure_mud_mortar_brick": 0,
  "has_superstructure_cement_mortar_brick": 0,
  "has_superstructure_timber": 0,
  "has_superstructure_bamboo": 0,
  "has_superstructure_rc_non_engineered": 0,
  "has_superstructure_rc_engineered": 0,
  "has_superstructure_other": 0,
  "legal_ownership_status": "v",
  "count_families": 1,
  "has_secondary_use": 0,
  "has_secondary_use_agriculture": 0,
  "has_secondary_use_hotel": 0,
  "has_secondary_use_rental": 0,
  "has_secondary_use_institution": 0,
  "has_secondary_use_school": 0,
  "has_secondary_use_industry": 0,
  "has_secondary_use_health_post": 0,
  "has_secondary_use_gov_office": 0,
  "has_secondary_use_use_police": 0,
  "has_secondary_use_other": 0
}

In [117]:
import pandas as pd
df = pd.DataFrame(ex, index=[0])
df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import numpy as np
geo3 = np.array(pd.get_dummies(df["geo_level_3_id"]))
geo3.shape

(1, 1)

In [7]:
from backend.api.encoder import Encoder
weights_path = "/home/joseph/Coding/ml_projects/earthquake_forecasting/models/geo_embed.h5"
enc = Encoder(weights_path)
enc.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 11861)]      0           []                               
                                                                                                  
 intermediate (Dense)           (None, 16)           189792      ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1418)         24106       ['intermediate[0][0]']           
                                                                                                  
 dense_1 (Dense)                (None, 31)           527         ['intermediate[0][0]']           
                                                                                              

In [15]:
import pandas as pd
train_path = "~/datasets/earthquake_damage_forecasting/train_values.csv"
train_df = pd.read_csv(train_path)
test_path = "~/datasets/earthquake_damage_forecasting/test_values.csv"
test_df = pd.read_csv(test_path)
train_df.head(), test_df.head()

(   building_id  geo_level_1_id  geo_level_2_id  geo_level_3_id  \
 0       802906               6             487           12198   
 1        28830               8             900            2812   
 2        94947              21             363            8973   
 3       590882              22             418           10694   
 4       201944              11             131            1488   
 
    count_floors_pre_eq  age  area_percentage  height_percentage  \
 0                    2   30                6                  5   
 1                    2   10                8                  7   
 2                    2   10                5                  5   
 3                    2   10                6                  5   
 4                    3   30                8                  9   
 
   land_surface_condition foundation_type  ... has_secondary_use_agriculture  \
 0                      t               r  ...                             0   
 1                      o 

In [118]:
len(train_df.columns)

39

In [121]:
# embeds:
embeds_path = "~/datasets/earthquake_damage_forecasting/train_data_embeds.csv"
e_df = pd.read_csv(embeds_path)
e_df

,Unnamed: 0,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,geo_feat7,geo_feat8,geo_feat9,geo_feat10,geo_feat11,geo_feat12,geo_feat13,geo_feat14,geo_feat15,geo_feat16
0,0,802906,2,30,6,5,1,1,0,0,...,-1.498751,-0.427753,1.313747,1.043078,2.243259,-0.485953,2.259708,1.000786,1.877474,0.547761
1,1,28830,2,10,8,7,0,1,0,0,...,0.876370,0.798046,1.537147,1.234688,0.813855,-1.614154,-0.076917,0.125310,1.477580,-1.249146
2,2,94947,2,10,5,5,0,1,0,0,...,-0.147367,1.463318,1.047525,0.996409,-0.107480,-1.332191,2.034211,2.696030,-0.663659,-1.964653
3,3,590882,2,10,6,5,0,1,0,0,...,-0.893961,0.917251,1.217506,0.874881,-0.263543,-1.850625,1.617405,-0.379536,-0.808820,-1.665377
4,4,201944,3,30,8,9,1,0,0,0,...,-0.548861,2.254541,0.001186,1.148989,1.442763,1.077754,0.544689,2.298833,2.032666,-2.213592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,260596,688636,1,55,6,3,0,1,0,0,...,0.249597,0.641817,0.734716,0.921014,1.718109,-0.918815,0.526678,1.720652,0.664152,-1.588144
260597,260597,669485,2,0,6,5,0,1,0,0,...,-1.128338,1.950719,0.041994,1.654622,1.238235,-1.453372,0.381435,-0.364326,1.842740,-2.540284
260598,260598,602512,3,55,6,7,0,1,0,0,...,0.694313,1.401268,0.308225,1.350058,1.412956,-0.686346,1.793258,-0.304019,0.722501,-2.049750
260599,260599,151409,2,10,14,6,0,0,0,0,...,-1.667232,2.649396,1.911566,1.281425,2.579090,-1.357294,1.040099,2.561289,0.871532,0.754798


In [125]:
train_df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,0,0,0,0
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,0,0,0,0


In [126]:
test_df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,310028,4,605,3623,3,70,20,6,t,r,...,1,0,0,0,0,0,0,0,0,0
86864,663567,10,1407,11907,3,25,6,7,n,r,...,0,0,0,0,0,0,0,0,0,0
86865,1049160,22,1136,7712,1,50,3,3,t,r,...,0,0,0,0,0,0,0,0,0,0
86866,442785,6,1041,912,2,5,9,5,t,r,...,0,0,0,0,0,0,0,0,0,0


In [123]:
e_df.columns

Index(['Unnamed: 0', 'building_id', 'count_floors_pre_eq', 'age',
       'area_percentage', 'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land_surface_condition_o', 'land_sur

In [124]:
cols = ['building_id', 'count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_stone_flag', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_t',
       'foundation_type_r', 'roof_type_n', 'ground_floor_type_f',
       'other_floor_type_q', 'position_t', 'plan_configuration_d',
       'legal_ownership_status_v', 'geo_feat1', 'geo_feat2', 'geo_feat3',
       'geo_feat4', 'geo_feat5', 'geo_feat6', 'geo_feat7', 'geo_feat8',
       'geo_feat9', 'geo_feat10', 'geo_feat11', 'geo_feat12', 'geo_feat13',
       'geo_feat14', 'geo_feat15', 'geo_feat16']
len(cols)

51

In [18]:
uniques = pd.concat([train_df["geo_level_3_id"], test_df["geo_level_3_id"]]).unique()
len(uniques)


11861

In [20]:
geo3 = np.array(pd.get_dummies(pd.concat([train_df["geo_level_3_id"], test_df["geo_level_3_id"]])))
geo3.shape

(347469, 11861)

In [26]:
classes = np.array(pd.concat([train_df["geo_level_3_id"], test_df["geo_level_3_id"]]))

In [61]:
np.unique(classes)

array([    0,     1,     2, ..., 12565, 12566, 12567])

In [71]:
classes[None].shape

(1, 347469)

In [77]:
classes

array([12198,  2812,  8973, ...,  7712,   912,  6436])

In [136]:
concat = pd.concat([train_df, test_df]).drop(["building_id"], axis=1)
concat.head()


,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [157]:
import joblib
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(concat["geo_level_3_id"])[:,None])
joblib.dump(enc, '../models/one_hot_geo.joblib')

['../models/one_hot_geo.joblib']

In [158]:
enc.categories_

[array([    0,     1,     2, ..., 12565, 12566, 12567])]

In [159]:
def drop_useless_cols(df: pd.DataFrame):
    """Specific function to drop columns not needed for one hot encoding"""
    if "building_id" in df.columns:
        df = df.drop(["building_id"], axis=1)

    if "Unnamed: 0" in df.columns:
        df = df.drop(["Unnamed: 0"], axis=1)
    df = df.drop(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], axis=1)
    return df

new_concat = drop_useless_cols(concat)
new_concat

,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,2,30,6,5,t,r,n,f,q,t,...,0,0,0,0,0,0,0,0,0,0
1,2,10,8,7,o,r,n,x,q,s,...,0,0,0,0,0,0,0,0,0,0
2,2,10,5,5,t,r,n,f,x,t,...,0,0,0,0,0,0,0,0,0,0
3,2,10,6,5,t,r,n,f,x,s,...,0,0,0,0,0,0,0,0,0,0
4,3,30,8,9,t,r,n,f,x,s,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,3,70,20,6,t,r,q,f,q,t,...,1,0,0,0,0,0,0,0,0,0
86864,3,25,6,7,n,r,n,f,q,s,...,0,0,0,0,0,0,0,0,0,0
86865,1,50,3,3,t,r,n,f,j,s,...,0,0,0,0,0,0,0,0,0,0
86866,2,5,9,5,t,r,n,f,q,s,...,0,0,0,0,0,0,0,0,0,0


In [170]:
one_hot = pd.get_dummies(new_concat)
len(one_hot.columns), one_hot.columns

(65,
 Index(['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
        'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
        'has_superstructure_stone_flag',
        'has_superstructure_cement_mortar_stone',
        'has_superstructure_mud_mortar_brick',
        'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
        'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
        'has_superstructure_rc_engineered', 'has_superstructure_other',
        'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
        'has_secondary_use_hotel', 'has_secondary_use_rental',
        'has_secondary_use_institution', 'has_secondary_use_school',
        'has_secondary_use_industry', 'has_secondary_use_health_post',
        'has_secondary_use_gov_office', 'has_secondary_use_use_police',
        'has_secondary_use_other', 'land_surface_condition_n',
        'land_surface_condition_o', 'land_surface_cond

In [187]:
one_hot

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,3,70,20,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86864,3,25,6,7,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
86865,1,50,3,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
86866,2,5,9,5,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [183]:
one_hot["area_percentage"]

0         6
1         8
2         5
3         6
4         8
         ..
86863    20
86864     6
86865     3
86866     9
86867    11
Name: area_percentage, Length: 347469, dtype: int64

In [186]:
one_hot["height_percentage"]

0        5
1        7
2        5
3        5
4        9
        ..
86863    6
86864    7
86865    3
86866    5
86867    4
Name: height_percentage, Length: 347469, dtype: int64

In [194]:
new_concat[new_concat.columns[0]].dtype == object

False

In [195]:
obj_cols = [col for col in new_concat.columns if new_concat[col].dtype == object]
obj_cols

['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [210]:
new_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347469 entries, 0 to 86867
Data columns (total 35 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   count_floors_pre_eq                     347469 non-null  int64 
 1   age                                     347469 non-null  int64 
 2   area_percentage                         347469 non-null  int64 
 3   height_percentage                       347469 non-null  int64 
 4   land_surface_condition                  347469 non-null  object
 5   foundation_type                         347469 non-null  object
 6   roof_type                               347469 non-null  object
 7   ground_floor_type                       347469 non-null  object
 8   other_floor_type                        347469 non-null  object
 9   position                                347469 non-null  object
 10  plan_configuration                      347469 non-null  

In [188]:
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 347469 entries, 0 to 86867
Data columns (total 65 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   count_floors_pre_eq                     347469 non-null  int64
 1   age                                     347469 non-null  int64
 2   area_percentage                         347469 non-null  int64
 3   height_percentage                       347469 non-null  int64
 4   has_superstructure_adobe_mud            347469 non-null  int64
 5   has_superstructure_mud_mortar_stone     347469 non-null  int64
 6   has_superstructure_stone_flag           347469 non-null  int64
 7   has_superstructure_cement_mortar_stone  347469 non-null  int64
 8   has_superstructure_mud_mortar_brick     347469 non-null  int64
 9   has_superstructure_cement_mortar_brick  347469 non-null  int64
 10  has_superstructure_timber               347469 non-null  int64
 11  h

In [223]:
new_concat[obj_cols]

,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status
0,t,r,n,f,q,t,d,v
1,o,r,n,x,q,s,d,v
2,t,r,n,f,x,t,d,v
3,t,r,n,f,x,s,d,v
4,t,r,n,f,x,s,d,v
...,...,...,...,...,...,...,...,...
86863,t,r,q,f,q,t,d,w
86864,n,r,n,f,q,s,d,v
86865,t,r,n,f,j,s,d,v
86866,t,r,n,f,q,s,d,a


In [221]:
enc = OneHotEncoder(handle_unknown='error', sparse=False)
enc.fit(new_concat[obj_cols])
joblib.dump(enc, '../models/one_hot.joblib')

['../models/one_hot.joblib']

In [216]:
len(enc.categories_)

8

In [217]:
enc.categories_

[array(['n', 'o', 't'], dtype=object),
 array(['h', 'i', 'r', 'u', 'w'], dtype=object),
 array(['n', 'q', 'x'], dtype=object),
 array(['f', 'm', 'v', 'x', 'z'], dtype=object),
 array(['j', 'q', 's', 'x'], dtype=object),
 array(['j', 'o', 's', 't'], dtype=object),
 array(['a', 'c', 'd', 'f', 'm', 'n', 'o', 'q', 's', 'u'], dtype=object),
 array(['a', 'r', 'v', 'w'], dtype=object)]

In [218]:
out = enc.transform(new_concat[obj_cols]).toarray()
out

array([[0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [222]:
enc.get_feature_names()

/home/joseph/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_n', 'x0_o', 'x0_t', 'x1_h', 'x1_i', 'x1_r', 'x1_u', 'x1_w',
       'x2_n', 'x2_q', 'x2_x', 'x3_f', 'x3_m', 'x3_v', 'x3_x', 'x3_z',
       'x4_j', 'x4_q', 'x4_s', 'x4_x', 'x5_j', 'x5_o', 'x5_s', 'x5_t',
       'x6_a', 'x6_c', 'x6_d', 'x6_f', 'x6_m', 'x6_n', 'x6_o', 'x6_q',
       'x6_s', 'x6_u', 'x7_a', 'x7_r', 'x7_v', 'x7_w'], dtype=object)

In [ ]:
out_df = pd.DataFrame(transformed, columns=jobs_encoder.get_feature_names())

In [219]:
len(new_concat.columns)-len(obj_cols) + 38

65

In [220]:
out.shape

(347469, 38)

In [110]:
df = train_df[0:1]
df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0


In [115]:
np.array(train_df[0:1]["geo_level_3_id"])[None].shape

(1, 1)

In [145]:
arr = np.zeros((1, 36))
arr[0] = np.array(train_df[0:1]["geo_level_3_id"])

In [146]:
enc.transform(arr)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GetDummies(BaseEstimator, TransformerMixin):
    def __init__(self, dummy_columns):
        self.columns = None
        self.dummy_columns = dummy_columns

    def fit(self, X, y=None):
        self.columns = pd.get_dummies(X, columns=self.dummy_columns).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X, columns=self.dummy_columns)
        return X_new.reindex(columns=self.columns, fill_value=0)

In [232]:
cols = list(e_df.drop(['Unnamed: 0', 'building_id', 'geo_feat1', 'geo_feat2', 'geo_feat3', 'geo_feat4', 'geo_feat5', 'geo_feat6',
       'geo_feat7', 'geo_feat8', 'geo_feat9', 'geo_feat10', 'geo_feat11',
       'geo_feat12', 'geo_feat13', 'geo_feat14', 'geo_feat15', 'geo_feat16'], axis=1).columns)

In [235]:
cols

['count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other',
 'land_surface_condition_n',
 'land_surface_condition_o',
 'land_surface_condition_t',
 'foundation_type_h',
 'foundation_type_i',
 'foundation_type_r',
 'foundation_type_u

In [245]:
# col = "plan_configuration"
# col = "other_floor_type"
col = "legal_ownership_status"
new_concat[0:1][col]

0    v
Name: legal_ownership_status, dtype: object

In [238]:
new_test = pd.get_dummies(new_concat[0:1]).reindex(columns=cols, fill_value=0)
new_test

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
0,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [240]:
new_test["plan_configuration_d"]

0    1
Name: plan_configuration_d, dtype: uint8

In [242]:
new_test["other_floor_type_q"]

0    1
Name: other_floor_type_q, dtype: uint8

In [244]:
new_test["other_floor_type_s"]

0    0
Name: other_floor_type_s, dtype: int64